In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from selenium import webdriver
import time
import requests

In [2]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['FACEBOOK_ACCESS_TOKEN']=os.getenv('FACEBOOK_ACCESS_TOKEN')
access_token = os.environ['FACEBOOK_ACCESS_TOKEN']
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
url = f"https://graph.facebook.com/debug_token?input_token={access_token}&access_token={access_token}"

response = requests.get(url)
data = response.json()

for i in data['data']['granular_scopes']:
    if i['scope']=='instagram_basic':
        ig_user_id=i['target_ids'][0]

In [4]:
def get_games(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)  # Adjust sleep time as needed for content to load
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    main_div = soup.find('main', id='main')
    links=["https://www.pff.com"+link.get('href')+'#recap' for link in main_div.find_all('a')]
    driver.quit()
    return links
    

In [5]:
def get_content(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)  # Adjust sleep time as needed for content to load
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    main_div = soup.find('main', id='main')
    if main_div:
        content_div=main_div.find_all('div',class_='m-content')[0].find_all('div',class_='m-longform-copy')[0]
        if content_div:
            content=content_div.get_text(separator=" ",strip=True)
            print('Content Gathered Successfully')
        else:
            print("m-content div not found within main div")
    else:
        print("Main div not found")
    
    driver.quit()
    return content

In [6]:
def create_haiku(content):
    system_prompt = (
    "You are an AI designed to craft haikus that capture the essence of NFL game recaps. "
    "Focus on identifying the key elements that define each game, such as dramatic moments, game-changing plays, "
    "and standout players. Use these highlights to create a 5-7-5 haiku format that is poetic and evocative, "
    "bringing the story of the game to life in just three lines. Ensure each haiku reflects the spirit and emotion of the game, "
     "while maintaining the syllable structure.")
    
    user_prompt=(
        "Create a Haiku from NFL game recaps."
        "Here is the game recap:"+content)
    
    user_prompt=user_prompt[0:20000]
        
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content":user_prompt}
          ],
    )
    result = response.choices[0].message.content
    return result

In [7]:
# def create_meme(content):
#     system_prompt = (
#         "You are a clever and witty meme creator with an understanding of NFL terminology."
#     )
#     user_prompt=(    
#     "Generate a humorous, text-based sports recap in the style of a meme. "
#     "Capture the key events of the game moment with clever phrasing, emphasizing any ironic or dramatic twists. "
#     "Use witty dialogue or reactions that express celebration, frustration, or surprise, depending on the context of the event. "
#     "Keep the text concise and engaging.")
    
#     response = openai.chat.completions.create(
#     model=MODEL,
#     messages=[
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content":user_prompt}
#       ],
#     )
#     result = response.choices[0].message.content
#     return result

In [13]:
def generate_image(meme):
    # Generate an image from a prompt
    prompt = (
        "Create a meme image of a sports moment with NO TEXT, focusing on players' and fans' reactions to the moment. Make sure that the team colors make sense for the NFL team color scheme: "
        + meme + 
        ". Show emotions visually, such as excitement, frustration, or surprise, with exaggerated expressions and body language. Do not include any text in the image at all."
    )

    # Ensure the prompt doesn't exceed the character limit
    prompt = prompt[0:1000]
    
    response = openai.images.generate(
        prompt=prompt,
        model="dall-e-3",
        n=1,  # number of images to generate
        size="1024x1024"  # specify image resolution (256x256, 512x512, or 1024x1024)
    )

    return response.data[0].url

In [14]:
def generate_creation_id(image_url,caption):
    image_url=image_url
    caption=caption
    upload_url = f"https://graph.facebook.com/v17.0/{ig_user_id}/media"
    upload_payload = {
        "image_url": image_url,
        "caption": caption,
        "access_token": access_token
    }
    upload_response = requests.post(upload_url, data=upload_payload)
    upload_data = upload_response.json()
    return upload_data["id"]
    

In [15]:
def publish_media(creation_id):
    # Publish media
    publish_url = f"https://graph.facebook.com/v17.0/{ig_user_id}/media_publish"
    publish_payload = {
        "creation_id": creation_id,
        "access_token": access_token
    }
    
    publish_response = requests.post(publish_url, data=publish_payload)
    publish_data = publish_response.json()
    print(publish_data)
    

In [16]:
games=get_games('https://www.pff.com/nfl/scores?week=12')

In [22]:
for game in games:
    try:
        content=get_content(game)
        haiku=create_haiku(content)
        image=generate_image(content)
        creation_id=generate_creation_id(image,haiku)
        publish_media(creation_id)
        print(game+' uploaded to ig')
    except:
        print('Game recap does not exist for : '+game)

Content Gathered Successfully
{'error': {'message': 'Application request limit reached', 'type': 'OAuthException', 'is_transient': False, 'code': 4, 'error_subcode': 2207051, 'error_user_title': 'action is blocked', 'error_user_msg': 'We restrict certain activity to protect our community. Tell us if you think we made a mistake.', 'fbtrace_id': 'AE9io1dIK_MaJjidmKyvHQr'}}
https://www.pff.com/nfl/scores/2024/12/pittsburgh-steelers_at_cleveland-browns_26060#recap uploaded to ig
Content Gathered Successfully
{'error': {'message': 'Application request limit reached', 'type': 'OAuthException', 'is_transient': False, 'code': 4, 'error_subcode': 2207051, 'error_user_title': 'action is blocked', 'error_user_msg': 'We restrict certain activity to protect our community. Tell us if you think we made a mistake.', 'fbtrace_id': 'AhHwCrbWrusZXy8i5yXwmCp'}}
https://www.pff.com/nfl/scores/2024/12/kansas-city-chiefs_at_carolina-panthers_26058#recap uploaded to ig
Content Gathered Successfully
{'error': 